# 🔹 LLM / Transformer için Search / Inference

LLM’lerde (T5, BART, GPT tarzı modeller) **token üretimi** decoder üzerinden adım adım yapılır.  
Bu sürece **search** veya **decoding strategy** denir. Farklı yöntemler model çıktısının kalitesini ve çeşitliliğini etkiler.

---

## 1️⃣ Greedy Decoding (En Basit)

**Tanım:**  
Her adımda en yüksek olasılıklı token seçilir.

**Özellikler:**
- Hızlı ve deterministik
- Tek bir çıktıya odaklı
- Çeşitlilik düşüktür, bazen daha az doğal metin üretir

**Adımlar:**
1. Encoder’dan çıktı alınır: `enc_out = model.encoder(encoder_input_ids)`
2. Decoder input başlatılır: BOS token
3. Her adımda:
   - Decoder logits üretir
   - Son token argmax ile seçilir
   - Decoder input güncellenir (`concat`)
4. EOS token geldiğinde veya max_length’e ulaşıldığında durur

**Avantaj:** Hızlı ve stabil  
**Dezavantaj:** Kreatif/çeşitli çıktılar düşük

---

## 2️⃣ Beam Search

**Tanım:**  
Her adımda **en iyi K sequence’i** takip eder. Olasılık score’u üzerinden seçim yapılır.

**Özellikler:**
- Daha doğru ve tutarlı çıktı üretir
- Hesaplama maliyeti biraz daha yüksektir
- Beam width → takip edilen alternatif dizilerin sayısı

**Adımlar:**
1. Encoder’dan çıktı alınır
2. Başlangıç: BOS token ile beam listesi oluşturulur
3. Her adımda:
   - Tüm beam’ler için logits hesaplanır
   - Top-K olasılıklı tokenlar seçilir
   - Score güncellenir (`-log(prob)`) → minimize loss
   - En iyi K sequence seçilir
4. EOS ile biten sequence kontrol edilir

**Avantaj:** Deterministik ve kaliteli metin üretir  
**Dezavantaj:** Beam width arttıkça hesaplama maliyeti artar

---

## 3️⃣ Sampling (Top-K / Top-p)

**Tanım:**  
Belirli bir olasılık dağılımından rastgele token seçilir.  

- **Top-K:** Sadece en olası K token arasından seçim  
- **Top-p (nucleus sampling):** Olasılık kümülatifi p’ye kadar tokenlar arasından seçim  

**Özellikler:**
- Daha yaratıcı ve doğal metin
- Bazen hatalı token seçimi olabilir
- Deterministik değildir

---

## 4️⃣ Decoder’a Self vs Cross Attention Aktarımı

LLM decoder katmanında genellikle iki attention vardır:

| Attention Türü | Kullanım |
|----------------|----------|
| Self-Attention  | Decoder kendi geçmiş tokenlarını görür |
| Cross-Attention | Encoder çıktısını okur (source sequence) |

**Uygulama:**
- Self-Attention her zaman çalışır
- Cross-Attention sadece `enc_out` sağlanırsa çalışır
- `if enc_out is not None:` koşulu ile aktif edilir

---

## 5️⃣ Özet: Hangi Yöntem Ne Zaman Kullanılır?

| Search Method | Avantaj | Dezavantaj | LLM Kullanım Önerisi |
|---------------|---------|------------|--------------------|
| Greedy | Hızlı, basit | Tekdüze | Basit çeviri, test |
| Beam Search | Tutarlı, doğru | Hesaplama | Seq2Seq çeviri, özetleme |
| Top-K / Top-p | Yaratıcı, doğal | Bazen hatalı | Yaratıcı metin üretimi |

---

💡 **Not:**  
Tüm yöntemler decoder’ın **step-by-step token üretimi** mantığına uygun olarak çalışır.  
Decoder input sürekli güncellenir, masking ve attention mask doğru şekilde uygulanmalıdır.


## 🔹  Hangileri kritik?

* Beam Search: Seq2Seq görevleri için temel ve vazgeçilmez

* Top-p / Top-K Sampling: Yaratıcı içerik için mutlaka bilinmeli

* Greedy: Hızlı test ve debug için pratik

🔑 Not: Decoder step-by-step çalıştığı için masking, attention ve input güncellemeleri tüm stratejilerde doğru uygulanmalıdır.